# Tutorial 1: A basic avatarization

In this tutorial, we will connect to a server to perform the avatarization of a dataset that does not require any pre-processing. We'll retrieve the anonymized dataset and the associated avatarization report. 

In [ ]:
import os
import secrets

import pandas as pd

from avatars.manager import Manager
from avatars.models import JobKind

url = os.environ.get("AVATAR_BASE_API_URL", "https://www.octopize.app/api")
username = os.environ.get("AVATAR_USERNAME", "")
password = os.environ.get("AVATAR_PASSWORD", "")

In [ ]:
manager = Manager(base_url=url)
# Authenticate with the server
manager.authenticate(username, password)

In [ ]:
# Verify that we can connect to the API server
manager.auth_client.health.get_health()

## Loading data

We recommend loading your file as a pandas dataframe. It enables you to check your data before avatarization and to pre-process it if required. 

In this tutorial, we use the simple and well-known `iris` dataset to demonstrate the main steps of an avatarization.

In [ ]:
df = pd.read_csv("../fixtures/iris.csv")

In [ ]:
df

In [ ]:
# The runner is the object that will be used to upload data to the server and run the avatarization
runner = manager.create_runner(f"iris_k5_{secrets.token_hex(4)}")

# Then upload the data, you can either use a pandas dataframe or a file
runner.add_table("iris", df)

## Creating and launching an avatarization job

In [ ]:
runner.set_parameters("iris", k=5)

In [ ]:
avatarization_job = (
    runner.run()
)  # by default we run all jobs : avatarization, privacy and signal metrics and report
# You can also choose to run only the avatarization job for example
# avatarization_job = runner.run(jobs_to_run=[JobKind.standard])

## Retrieving the avatars

In [ ]:
runner.shuffled("iris").head()

## Retrieving the privacy metrics

In [ ]:
runner.privacy_metrics("iris")[0]

## Retrieving the signal metrics

In [ ]:
runner.signal_metrics("iris")[0]

# Download the report

In [ ]:
runner.download_report("my_report.pdf")

# How to print an error message 
There are multiple types of error and we encourage you to have a look at our [documentation](https://python.docs.octopize.io/latest/user_guide.html#understanding-errors) to understand them.

The most common error is when server validation prevents a job from running.

The following section show how to print an error message. 

In [ ]:
runner = manager.create_runner(f"iris_fail_{secrets.token_hex(4)}")
runner.add_table("iris", df)

runner.set_parameters("iris", k=500)  # k is too big (bigger than the dataset !)

runner.run(jobs_to_run=[JobKind.standard])

In [ ]:
error_job = runner.get_job(JobKind.standard)
print(error_job.status)
print(error_job.exception)